In [1]:
import torch
from torch import nn, optim
import torchvision
import torch.nn.functional as F
import torchvision.datasets as datasets
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
import matplotlib.pyplot as plt
from torchvision.transforms import ToPILImage
%matplotlib inline
import time
show = ToPILImage() #可以把Tensor转换成Image,方便可视化
transform = transforms.Compose([  #transforms.Compose就是将对图像处理的方法集中起来
    transforms.RandomHorizontalFlip(),#水平翻转
    transforms.RandomCrop((32, 32), padding=4),#对图片进行随机裁剪，裁剪的大小是32*32的，填充是4
    transforms.ToTensor(),#转为Tensor
    #在做数据归一化之前必须要把PIL Image转成Tensor，而其他resize或crop操作则不需要。
     transforms.Normalize((0.5, 0.5 ,0.5), (0.5, 0.5, 0.5)),])

# 超参数定义
EPOCH = 300               # 训练epoch次数
BATCH_SIZE = 200         # 批训练的数量
LR = 0.0001              # 学习率
DOWNLOAD_MNIST = False  # 设置True 可以自动下载数据

# CIFAR-10数据集下载
train_data = datasets.CIFAR10(root='/Users/yangkaixiang/kkb/24-7/cifar-10-batches-py',
                         train=True,                         # 训练集
                         # 数据变换(0, 255) -> (0, 1)
                         transform=transform,
                         download=True     
                           )

test_data = datasets.CIFAR10(root='/Users/yangkaixiang/kkb/24-7/cifar-10-batches-py',
                        train=False,                         # 测试集
                        transform=transform,
                        download=True
                          )

# 使用DataLoader进行分批
train_loader = DataLoader(dataset=train_data, batch_size=BATCH_SIZE, shuffle=True)
test_loader = DataLoader(dataset=test_data, batch_size=BATCH_SIZE, shuffle=True)

#数据集10个类的定义
classes = ('plane', 'car', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

# densenet121 Model
model = torchvision.models.densenet121(pretrained=True)
#损失函数:这里用交叉熵
criterion = nn.CrossEntropyLoss()
#优化器 Adam
optimizer = optim.Adam(model.parameters(), lr=1e-4)

#device : GPU or CPU
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model.to(device)

print('Start Training')
# 训练

for epoch in range(EPOCH):
    running_loss = 0.0
    start = time.time()
    for i, data in enumerate(train_loader):#enumerate枚举数据并从下标0开始
        # 读取数据的数据内容和标签
        #print('读取数据的数据内容和标签')
        inputs, labels = data
        inputs, labels = inputs.to(device), labels.to(device)
        # 前向传播
        #print('前向传播')
        outputs = model(inputs)
        # 计算损失函数
        #print('计算损失函数')
        loss = criterion(outputs, labels)
        #梯度清零，也就是把loss关于weight的导数变成0.
        optimizer.zero_grad()
        # 反向传播
        loss.backward()
        # 参数更新
        optimizer.step()
    end = time.time()
    print('epoch{} loss:{:.4f} using time is {:.2f}s'.format(epoch+1, loss.item(), (end-start)))
    if loss.item() < 0.001:
        print('Training Stop')
        break
print("Finished Traning")

Extracting /Users/yangkaixiang/kkb/24-7/cifar-10-batches-py/cifar-10-python.tar.gz to /Users/yangkaixiang/kkb/24-7/cifar-10-batches-py
Files already downloaded and verified


Downloading: "https://download.pytorch.org/models/densenet121-a639ec97.pth" to /Users/yangkaixiang/.cache/torch/hub/checkpoints/densenet121-a639ec97.pth



Start Training
epoch1 loss:0.9641 using time is 1351.69s
epoch2 loss:0.8943 using time is 1242.30s
epoch3 loss:0.7421 using time is 1257.48s
epoch4 loss:0.5263 using time is 1397.60s
epoch5 loss:0.5176 using time is 1259.20s
epoch6 loss:0.5315 using time is 1242.01s
epoch7 loss:0.5873 using time is 1286.35s
epoch8 loss:0.3354 using time is 2438.99s
epoch9 loss:0.3090 using time is 1256.48s
epoch10 loss:0.1857 using time is 1238.17s
epoch11 loss:0.2662 using time is 1236.37s
epoch12 loss:0.2470 using time is 1239.95s
epoch13 loss:0.2450 using time is 1293.61s
epoch14 loss:0.2433 using time is 1271.65s
epoch15 loss:0.2295 using time is 6681.71s
epoch16 loss:0.1845 using time is 1298.11s
epoch17 loss:0.1734 using time is 1308.46s
epoch18 loss:0.1724 using time is 1278.99s
epoch19 loss:0.1718 using time is 1262.14s
epoch20 loss:0.1215 using time is 1305.79s
epoch21 loss:0.1098 using time is 1270.92s
epoch22 loss:0.1051 using time is 1308.82s
epoch23 loss:0.0918 using time is 1358.77s
epoc

KeyboardInterrupt: 

In [2]:
# 测试
model.eval()
correct = 0
total = 0

for data in test_loader:
    images, labels = data
    images, labels = images.to(device), labels.to(device)
    # 前向传播
    out = model(images)
    _, predicted = torch.max(out.data, 1)
    total += labels.size(0)
    correct += (predicted == labels).sum().item()

#输出识别准确率
print('10000测试图像 准确率:{:.4f}%'.format(100 * correct / total)) 

10000测试图像 准确率:85.8900%
